In [16]:
import pandas as pd
!pip install polars
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pl.read_csv(r"C:\Users\jj_jo\Documents\Customer churn project.csv")
df


Call Failure,Complaints,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64
8,0,38,0,4370,71,5,17,3,1,1,30,197.64,0
0,0,39,0,318,5,7,4,2,1,2,25,46.035,0
10,0,37,0,2453,60,359,24,3,1,1,30,1536.52,0
10,0,38,0,4198,66,1,35,1,1,1,15,240.02,0
3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0
11,0,38,1,3775,82,32,28,3,1,1,30,282.28,0
4,0,38,0,2360,39,285,18,3,1,1,30,1235.96,0
13,0,37,2,9115,121,144,43,3,1,1,30,945.44,0
7,0,38,0,13773,169,0,44,3,1,1,30,557.68,0


In [17]:
# Which age groups send more SMS messages than make phone calls?
# gathering all data I need

age_group_data = df.group_by("Age Group").agg(
    [pl.col("Frequency of SMS").sum(), pl.col("Frequency of use").sum(), pl.col("Age").mean()]
)

age_group_data = age_group_data.sort("Frequency of SMS", descending = True)
age_group_data

# age group 3 and 2 sends more SMS than make phone calls

Age Group,Frequency of SMS,Frequency of use,Age
i64,i64,i64,f64
3,128311,97531,30.0
2,78293,74742,25.0
4,16611,23922,45.0
5,4802,13179,55.0
1,2484,9427,15.0


In [18]:
# compute difference in percentage
age_group_data = age_group_data.rename({"Frequency of use": "Frequency of phone calls"})

age_group_data = age_group_data.with_columns(
    ((pl.col("Frequency of SMS") - pl.col("Frequency of phone calls")) / pl.col("Frequency of phone calls") * 100).alias("percent change")
    
)

age_group_data

# This shows that people in age group 3 send 31% more SMS than they make phone calls
# People in age group 2 send 4.7% more SMS than they make phone calls
# On the other hand, people in age group 4 make 30% more phone calls than they send SMS

Age Group,Frequency of SMS,Frequency of phone calls,Age,percent change
i64,i64,i64,f64,f64
3,128311,97531,30.0,31.559197
2,78293,74742,25.0,4.75101
4,16611,23922,45.0,-30.561826
5,4802,13179,55.0,-63.563245
1,2484,9427,15.0,-73.650154


In [19]:
# find out average age in each group
#age_group_data = age_group_data.rename({"Age": "Avg age"})
age_group_data

# We can see age group 3 and 2 use SMS more than they make phone calls
# The average age in these two groups are 30 and 25

Age Group,Frequency of SMS,Frequency of phone calls,Age,percent change
i64,i64,i64,f64,f64
3,128311,97531,30.0,31.559197
2,78293,74742,25.0,4.75101
4,16611,23922,45.0,-30.561826
5,4802,13179,55.0,-63.563245
1,2484,9427,15.0,-73.650154


In [20]:
# Create a plot visualizing the number of distinct phone calls by age group. 
# Within the chart, differentiate between short, medium, and long calls (by the number of seconds).
# find mean of seconds of use
# find std of seconds of use
# mean - std can be used to compare with the original seconds of use values

mean = df.select(pl.col("Seconds of Use")).mean()
std = df.select(pl.col("Seconds of Use")).std()



df = df.with_columns(
    pl.when(pl.col("Seconds of Use") < mean - std).then("Short")
    .when((mean - std <= pl.col("Seconds of Use")) & (pl.col("Seconds of Use") <= mean + std)).then("Medium")
    .when(pl.col("Seconds of Use") >= mean + std).then("Long")
    .alias("Call length")
)

df 



C:\Users\jj_jo\AppData\Local\Temp\ipykernel_3304\1805320262.py:13: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('Short')`
  pl.when(pl.col("Seconds of Use") < mean - std).then("Short")
C:\Users\jj_jo\AppData\Local\Temp\ipykernel_3304\1805320262.py:14: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('Medium')`
  .when((mean - std <= pl.col("Seconds of Use")) & (pl.col("Seconds of Use") <= mean + std)).then("Medium")
C:\Users\jj_jo\AppData\Local\Temp\ipykernel_3304\1805320262.py:15: DeprecationWarning: in a future version, string input will be parsed as a column name rather than a string literal. To silence this warning, pass the input as an expression instead: `pl.lit('Long')`
  .when(pl.col("Seconds of U

Call Failure,Complaints,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn,Call length
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str
8,0,38,0,4370,71,5,17,3,1,1,30,197.64,0,"""Medium"""
0,0,39,0,318,5,7,4,2,1,2,25,46.035,0,"""Medium"""
10,0,37,0,2453,60,359,24,3,1,1,30,1536.52,0,"""Medium"""
10,0,38,0,4198,66,1,35,1,1,1,15,240.02,0,"""Medium"""
3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0,"""Medium"""
11,0,38,1,3775,82,32,28,3,1,1,30,282.28,0,"""Medium"""
4,0,38,0,2360,39,285,18,3,1,1,30,1235.96,0,"""Medium"""
13,0,37,2,9115,121,144,43,3,1,1,30,945.44,0,"""Long"""
7,0,38,0,13773,169,0,44,3,1,1,30,557.68,0,"""Long"""


In [21]:
# Create a plot visualizing the number of distinct phone calls by age group. 
# Within the chart, differentiate between short, medium, and long calls (by the number of seconds).
plot_graph = df.select(
    pl.col("Age Group"),
    pl.col("Distinct Called Numbers"),
    pl.col("Call length")

)

plot_graph

Age Group,Distinct Called Numbers,Call length
i64,i64,str
3,17,"""Medium"""
2,4,"""Medium"""
3,24,"""Medium"""
1,35,"""Medium"""
1,33,"""Medium"""
3,28,"""Medium"""
3,18,"""Medium"""
3,43,"""Long"""
3,44,"""Long"""


In [22]:
# group the age group and call length by distinct called numbers
plot_graph = df.group_by("Age Group").agg(
    pl.col("Distinct Called Numbers").sum(),
    pl.col("Call length").mode()
    
)

plot_graph


Age Group,Distinct Called Numbers,Call length
i64,i64,list[str]
3,30641,"[""Medium""]"
4,10304,"[""Medium""]"
1,4222,"[""Medium""]"
5,5053,"[""Medium""]"
2,23836,"[""Medium""]"


In [24]:
plot_graph
sns.barplot(plot_graph.to_pandas(), x = "Age Group", y = "Distinct Called Numbers",hue = "Call length")
plt.title("Num of distinct phone calls by age group")

# Couldn't get this to work in polars no matter how much I researched
# For some reason hue paremeter doesn't work in polars. If I remove hue, the graph works. 
# Solutions all say to just convert the graph to pandas and it works but couldn't get it to work


TypeError: unhashable type: 'numpy.ndarray'

In [25]:
# Are there significant differences between the length of phone calls between different tariff plans?
tariff_plan_data = df.group_by("Tariff Plan").agg(
    pl.col("Seconds of Use").mean()
)

tariff_plan_data
# Avg call length is longer on Tariff plan 2


Tariff Plan,Seconds of Use
i64,f64
2,6403.261224
1,4309.620998


In [26]:
tariff_plan_data.with_columns(
    (pl.col("Seconds of Use").pct_change() * 100).alias("pct change")

)

# Avg call length is longer by approx 48%

Tariff Plan,Seconds of Use,pct change
i64,f64,f64
2,6403.261224,null
1,4309.620998,-32.696468


In [27]:
# What effects Churn
churn_data = df.group_by("Churn").agg(
    pl.col("Call Failure").mean(),
    pl.col("Complaints").count()
)

churn_data



Churn,Call Failure,Complaints
i64,f64,u32
1,7.476768,495
0,7.656121,2655


In [28]:
churn_data = churn_data.with_columns(
    (pl.col("Call Failure").pct_change() * 100).alias("Call Failure %"),
    (pl.col("Complaints").pct_change() * 100).alias("Complaints %")
)

churn_data
# Those who don't Churn experience 2.39% more Call failures on avg and 436% more complaints
# Shows no positive correlation between Churning and call failure/ complaints


Churn,Call Failure,Complaints,Call Failure %,Complaints %
i64,f64,u32,f64,f64
1,7.476768,495,null,null
0,7.656121,2655,2.398802,436.363636
